# Deterministic optimisation

Here we optimise over several different objective functions evaluated using the deterministic fluid flow model. 

In [1]:
# imports
import deterministic_optimisation as do

In [2]:
# constraint funcs
def budget_constraint(problem):
    costs=0
    for t in problem.T:
        costs += problem.h[t] * costs_accomm['housing']
        costs += problem.s[t] * costs_accomm['shelter']
    return costs <= budget

def min_house_build(problem,n):
    return problem.h[n]>=baseline_build

def min_shelter_build(problem,n):
    return problem.s[n]>=baseline_build

In [3]:
# Set as is data
data_as_is = {'initial_capacity' : {'housing':40, 'shelter':15},
              'initial_demand' : 120,
              'service_mean' : {'housing': (0+6+8)/3, 'shelter': 0.0},
              'arrival_rates' : [36.56, 43.52, 47.77, 47.77, 43.13]}

# Model setup
horizon = 5
timestep = 1/365
budget = 2000
baseline_build = 5
costs_accomm = {'housing' : 30, 'shelter' : 10}
model = do.FluidModel

####  Phi 0
    # min TimeAvg(E[unsh(t)])
    # s.t. total budget constraint
    #      annual minimum build constraint

In [4]:
def y0(problem):
    " objective function for problem Phi0 "
    solution = {'housing' : problem.h, 'shelter' : problem.s}
    fluid_model = model(data_as_is, solution)
    fluid_model.analyse(horizon, timestep)
    avg_unsh = sum(fluid_model.model.unsh_t)/len(fluid_model.model.unsh_t)
    return(avg_unsh)

# Set up problem and solve
problem = do.Phi(data_as_is, 
                  timestep, 
                  horizon, 
                  budget, 
                  costs_accomm, 
                  baseline_build, 
                  budget_constraint, 
                  min_house_build, 
                  min_shelter_build, 
                  y0)

problem.solve('glpk')

# Outputs
print('House building solution per year: ' + str([round(i,2) for i in problem.h_opt]))
print('Shelter building solution per year: ' + str([round(i,2) for i in problem.s_opt]))
print('Optimal objective Val: ' + str(round(problem.instance.OBJ(),2)))

House building solution per year: [5.0, 5.0, 5.0, 5.0, 5.0]
Shelter building solution per year: [105.0, 5.0, 5.0, 5.0, 5.0]
Optimal objective Val: 30.0


Here we see that for the given cost of shelter and given housing service rate, it is best to spend all of our surplus budget on shelter in the first year. 

####  Phi 1
    # min TimeAvg(E[unsh(t) + c*E[sh(t)]])
    # s.t. total budget constraint
    #      annual minimum build constraint

In [5]:
def y1(problem):
    " objective function for problem Phi1 "
    solution = {'housing' : problem.h, 'shelter' : problem.s}
    fluid_model = model(data_as_is, solution)
    fluid_model.analyse(horizon, timestep)
    avg_unsh = sum(fluid_model.model.unsh_t)/len(fluid_model.model.unsh_t)
    avg_sh = sum(fluid_model.model.sh_t)/len(fluid_model.model.sh_t)
    c = 0.5 # weight placed on sheltered queue compared to unsheltered
    return(avg_unsh + (c * avg_sh))

# Set up problem and solve
problem = do.Phi(data_as_is, 
                  timestep, 
                  horizon, 
                  budget, 
                  costs_accomm, 
                  baseline_build, 
                  budget_constraint, 
                  min_house_build, 
                  min_shelter_build, 
                  y1)

problem.solve('glpk')

# Outputs
print('House building solution per year: ' + str([round(i,2) for i in problem.h_opt]))
print('Shelter building solution per year: ' + str([round(i,2) for i in problem.s_opt]))
print('Optimal objective Val: ' + str(round(problem.instance.OBJ(),2)))

House building solution per year: [5.0, 5.0, 5.0, 5.0, 5.0]
Shelter building solution per year: [105.0, 5.0, 5.0, 5.0, 5.0]
Optimal objective Val: 88.73


Here we see that by penalising the size of the sheltered queue, our preference shifts to spending all of our surplus budget on housing. 

####  Phi 2
    # min TimeAvg(E[unsh(t)^2] + c*E[sh(t)^2])
    # s.t. total budget constraint
    #      annual minimum build constraintint

In [6]:
def y2(problem):
    " objective function for problem Phi2 "
    solution = {'housing' : problem.h, 'shelter' : problem.s}
    fluid_model = model(data_as_is, solution)
    fluid_model.analyse(horizon, timestep)
    avg_unsh_2 = sum(fluid_model.model.unsh_sq_t)/len(fluid_model.model.unsh_sq_t)
    avg_sh_2 = sum(fluid_model.model.sh_sq_t)/len(fluid_model.model.sh_sq_t)
    c = 0.5 # weight placed on the penalty of sheltered queue compared to unsheltered
    return(avg_unsh_2 + (c * avg_sh_2))

# Set up problem and solve
problem = do.Phi(data_as_is, 
                  timestep, 
                  horizon, 
                  budget, 
                  costs_accomm, 
                  baseline_build, 
                  budget_constraint, 
                  min_house_build, 
                  min_shelter_build, 
                  y2)

problem.solve('ipopt')

# Outputs
print('House building solution per year: ' + str([round(i,2) for i in problem.h_opt]))
print('Shelter building solution per year: ' + str([round(i,2) for i in problem.s_opt]))
print('Optimal objective Val: ' + str(round(problem.instance.OBJ(),2)))

House building solution per year: [27.88, 5.0, 5.0, 5.0, 5.0]
Shelter building solution per year: [36.37, 5.0, 5.0, 5.0, 5.0]
Optimal objective Val: 6026.7
